In [6]:
import pandas as pd
import requests
import json
import datetime

from gnews import GNews
from newspaper import Article

## Defining Functions

In [7]:
def extract_article_newspaper(url):
    try:
        response = requests.get(url, timeout=15)  # will give up if it takes longer than 15 sec
        response.raise_for_status()
        if response.status_code == 200:
            
            article = Article(url)
            article.download()
            article.parse()
            content = article.text
    
            #article.nlp()
            #keywords = article.keywords
            print("article collected")
            #return content, keywords
            return content
        
        else:
            print(f"Failed to fetch URL: {url}")
            return None
        
    except requests.Timeout:
        print(f"Timeout occurred while fetching URL: {url}")
        return None
    
    except requests.RequestException as e:
        print(f"Error occurred while fetching URL: {url}. Error: {e}")
        return None
    
    except Exception as ex:
        print(f"Error occurred during article extraction for URL: {url}. Error: {ex}")
        return None

In [8]:
def append_to_csv(data, filename):
    try:
        existing_df = pd.read_csv(filename)
    
        # append the new data to the existing one
        updated_df = pd.concat([existing_df, data], ignore_index=True)
        updated_df.drop_duplicates(inplace=True)
    except FileNotFoundError:
        updated_df = data
        
    #write to same filename
    updated_df.to_csv(filename, index=False)

In [9]:
def pull_data(keyword, start_date, end_date):
    
    google_news = GNews(language='en', country='US', start_date=start_date, end_date=end_date)
    
    news = google_news.get_news(keyword)
    
    #change from json to dataframe
    news_df = pd.DataFrame(news)

    # remove 'href' part of the publisher column and rename it to source
    news_df['source'] = news_df['publisher'].apply(lambda x: x['title'])

    # drop the original publisher column
    news_df.drop(columns=['publisher'], inplace=True)
    
    # pull content using teh extract_article_newspaper function
    news_df['extracted_content'] = ''
    news_df['extracted_content'] = news_df['url'].apply(extract_article_newspaper)
    
    # dropping na values
    news_df = news_df.dropna(subset=['extracted_content'])
    
    # append any new results to the econ_news.csv using the append_to_csv function
    filename= 'data/econ_news.csv'
    append_to_csv(news_df, filename)
    
    print(f"results saved to {filename}")

## collecting news from conservative sources in particular

In [27]:
from newsapi import NewsApiClient

def pull_data_from_sources(keyword):
    #using news api instead to pull from specific sources
    newsapi = NewsApiClient(api_key='7604ff87bc484c31aa285e447a433fde')
    #sources = ['foxnews.com,foxbusiness.com,theamericanconservative.com,dailywire.com,thefederalist.com,washingtontimes.com,washingtonexaminer.com,nypost.com,nytimes.com,]
    domains = ['foxnews.com','foxbusiness.com','theamericanconservative.com','theblaze.com','dailywire.com','thefederalist.com','washingtontimes.com','washingtonexaminer.com','nypost.com','msnbc.com','washingtonpost.com','cnn.com','axios.com','newyorker.com','bloomberg.com','theatlantic.com']
    for website in domains:
        all_articles = newsapi.get_everything(q=keyword,
                                          domains=website,
                                          from_param='2024-04-01',
                                          to='2024-04-30',
                                          language='en',
                                          sort_by='relevancy')
                        
    return all_articles
    


In [28]:
website_articles = pull_data_from_sources('economy')
website_articles

{'status': 'ok',
 'totalResults': 28,
 'articles': [{'source': {'id': None, 'name': 'The Atlantic'},
   'author': 'Frank Bruni',
   'title': 'The New American Pessimism',
   'description': 'Too many Americans feel that our best days are behind us.',
   'url': 'https://www.theatlantic.com/ideas/archive/2024/04/the-age-of-grievance-frank-bruni/678127/?utm_source=feed',
   'urlToImage': None,
   'publishedAt': '2024-04-22T11:31:00Z',
   'content': 'It had been clear for years that China was rising and rising—building rail lines and airports and skyscrapers at a rate that put the United States to shame, purchasing the favor of poorer countries, … [+9812 chars]'},
  {'source': {'id': None, 'name': 'The Atlantic'},
   'author': 'Joseph E. Stiglitz',
   'title': 'Freedom for the Wolves',
   'description': 'Neoliberal orthodoxy holds that economic freedom is the basis of every other kind. That orthodoxy, a Nobel economist says, is not only false; it is devouring itself.',
   'url': 'https://ww

In [29]:
website_articles_df = pd.json_normalize(website_articles['articles'])
website_articles_df 

,author,title,description,url,urlToImage,publishedAt,content,source.id,source.name
0,Frank Bruni,The New American Pessimism,Too many Americans feel that our best days are...,https://www.theatlantic.com/ideas/archive/2024...,None,2024-04-22T11:31:00Z,It had been clear for years that China was ris...,None,The Atlantic
1,Joseph E. Stiglitz,Freedom for the Wolves,Neoliberal orthodoxy holds that economic freed...,https://www.theatlantic.com/ideas/archive/2024...,None,2024-04-22T11:00:00Z,Any discussion of freedom must begin with a di...,None,The Atlantic
2,Eric Schlosser,Do We Really Want a Food Cartel?,Mergers and acquisitions have created food oli...,https://www.theatlantic.com/ideas/archive/2024...,None,2024-04-09T14:55:44Z,The Federal Trade Commission has just released...,None,The Atlantic
3,Peter Wehner,Trump’s Willing Accomplice,Chris Sununu just showed how deep into the Rep...,https://www.theatlantic.com/politics/archive/2...,None,2024-04-15T17:58:13Z,"Yesterday, ABC News’s George Stephanopoulos co...",None,The Atlantic
4,Ilana Masad,Six Cult Classics You Have to Read,As word of mouth about a cult classic spreads ...,https://www.theatlantic.com/books/archive/2024...,None,2024-04-11T16:00:00Z,A book that earns the title of “cult classic” ...,None,The Atlantic
5,Eliot A. Cohen,Kyiv’s Grim Spring,"If the United States doesn’t do anything, the ...",https://www.theatlantic.com/international/arch...,None,2024-04-10T10:00:00Z,"Kyiv is, as ever, a lovely city, made more so ...",None,The Atlantic
6,Annalee Newitz,How Sci-Fi Inspired Conspiracy Theory,It all goes back to one man in the 1950s: a mi...,https://www.theatlantic.com/ideas/archive/2024...,None,2024-04-28T12:00:00Z,"In 1950, a U.S. Army psyops officer named Paul...",None,The Atlantic
7,Christopher Beam,There’s No Such Thing as a Price Anymore,The ubiquitous rise of add-on fees and persona...,https://www.theatlantic.com/ideas/archive/2024...,None,2024-04-16T11:00:00Z,"On February 15, Ron Ruggless was sitting in hi...",None,The Atlantic
8,Tyler Austin Harper,An Utterly Misleading Book About Rural America,White Rural Rage has become a best-seller—and ...,https://www.theatlantic.com/ideas/archive/2024...,None,2024-04-04T15:30:00Z,Rage is the subject of a new book by the polit...,None,The Atlantic
9,Hanna Rosin,"Money Can Buy You Everything, Except Maybe a B...",Unless you know how to play the “Hermès Game”,https://www.theatlantic.com/podcasts/archive/2...,None,2024-04-11T10:00:00Z,"Earlier this year, two California residents fi...",None,The Atlantic


In [281]:
right_articles_df['extracted_content'] = ''
right_articles_df['extracted_content'] = right_articles_df['url'].apply(extract_article_newspaper)

article collected
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://thefederalist.com/2024/04/23/how-chinas-flood-of-cheap-exports-boosts-its-economy-and-kills-ours/. Error: 403 Client Error: Forbidden for url: https://thefederalist.com/2024/04/23/how-chinas-flood-of-cheap-exports-boosts-its-economy-and-kills-ours/
Error occurred while fetching URL: https://thefederalist.com/2024/04/17/persistent-bidenflation-explains-why-americans-think-the-economy-is-poor/. Error: 403 Client Error: Forbidden for url: https://thefederalist.com/2024/04/17/persistent-bidenflation-explains-why-americans-think-the-economy-is-poor/
article collected
article collected
Error occurred while fetching URL: https://thefederalist.com/2024/04/11/margaret-sullivan-complains-voters-are-too-worried-about-the-economy-instead-of-democracy/. Error: 403 Client Error: Forbidden for url: https://thefederalist.com/2024/04/11/margaret-sullivan

article collected
article collected
article collected
Error occurred while fetching URL: https://www.theamericanconservative.com/mexicos-presidential-election-will-not-end-the-nations-crisis/. Error: 403 Client Error: Forbidden for url: https://www.theamericanconservative.com/mexicos-presidential-election-will-not-end-the-nations-crisis/
Error occurred while fetching URL: https://www.theamericanconservative.com/u-s-officials-believe-that-we-are-at-war-with-russia/. Error: 403 Client Error: Forbidden for url: https://www.theamericanconservative.com/u-s-officials-believe-that-we-are-at-war-with-russia/
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected


In [283]:
# adding conservative news to econ news dataset
append_to_csv(right_articles_df, 'data/econ_news.csv')

## Pulling  usign GNews

In [30]:
keywords= ['economy','finance','financial','trump economy','biden economy']

for year in range(2016, 2024):
    start_date = datetime.date(year, 1, 1)
    end_date = datetime.date(year, 12, 31)
    
    for topic in keywords:
        pull_data(topic, start_date, end_date)

article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiPWh0dHBzOi8vd3d3LmJscy5nb3YvYmRtL2VudHJlcHJlbmV1cnNoaXAvZW50cmVwcmVuZXVyc2hpcC5odG3SAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.bls.gov/bdm/entrepreneurship/entrepreneurship.htm
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMic2h0dHBzOi8vd3d3LmFtZXJpY2FucHJvZ3Jlc3Mub3JnL2FydGljbGUvc3RyZW5ndGhlbmluZy10aGUtcGFsZXN0aW5pYW4tZWNvbm9teS10by1rZWVwLWEtdHdvLXN0YXRlLXNvbHV0aW9uLXZpYWJsZS_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden

article collected
article collected
article collected
article collected
results saved to data/econ_news.csv
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMicWh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMTYvMTAvMjEveW91ci1tb25leS9pbS0yOS1hbmQtaS1uZXZlci1sZWFybmVkLWhvdy1tb25leS13b3Jrcy1pdHMtdGltZS10by1maXgtdGhhdC5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.nytimes.com/2016/10/21/your-money/im-29-and-i-never-learned-how-money-works-its-time-to-fix-that.html
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiQ2h0dHBzOi8vcm1pLm9yZy93YW50LXdvcmstc3VzdGFpbmFibGUtZmluYW5jZS1ybWktcHJvdmlkZXMtaW5zaWdodC_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://rmi.org/want-work-sustainable-finance-rmi-provides-insight/
article collected
article c

article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMid2h0dHBzOi8vd3d3LmZ1bGxzYWlsLmVkdS9hYm91dC9mdWxsLXNhaWwtc3Rvcmllcy9saW5kc2F5LW1lc2NoZXItZGlyZWN0b3Itb2YtcHVibGlzaGluZy1hbmQtZmluYW5jZS1hdC1uZXctd2VzdC1yZWNvcmRz0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.fullsail.edu/about/full-sail-stories/lindsay-mescher-director-of-publishing-and-finance-at-new-west-records
article collected
results saved to data/econ_news.csv
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiUmh0dHBzOi8vd3d3LmZiaS5nb3YvbmV3cy9zdG9yaWVzL2lyYW5pYW5zLWNoYXJnZWQtd2l0aC1oYWNraW5nLXVzLWZpbmFuY2lhbC1zZWN0b3LSAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidde

article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiUWh0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9zdG9yeS8yMDE2LzExL2tydWdtYW4tdHJ1bXAtZ2xvYmFsLXJlY2Vzc2lvbi0yMDE2LTIzMTA1NdIBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/story/2016/11/krugman-trump-global-recession-2016-231055
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiO2h0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9zdG9yeS8yMDE2LzAxL3RydW1wLWVjb25vbXktMjE3NDk20gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/story/2016/01/trump-economy-217496
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiTWh0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9zdG9yeS8yMDE2LzA5L2RvbmFsZC10cnVtcC1qb2JzLWVjb25vbWljLXBsYW4tMjI4MjE40gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 

article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiVGh0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9zdG9yeS8yMDE2LzAzL2Vjb25vbWlzdC10cnVtcC1wcmVzaWRlbmN5LWdsb2JhbC1yaXNrLTIyMDg4N9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/story/2016/03/economist-trump-presidency-global-risk-220887
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMid2h0dHA6Ly93d3cud3NqLmNvbS9hcnRpY2xlcy90cnVtcC1uYW1lcy1nb2xkbWFuLXByZXNpZGVudC1nYXJ5LWNvaG4tYXMtZGlyZWN0b3Itb2YtbmF0aW9uYWwtZWNvbm9taWMtY291bmNpbC0xNDgxNTczMDgy0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.wsj.com/articles/trump-names-goldman-president-gary-cohn-as-director-of-national-economic-council-1481573082
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiamh0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9tYWdhemluZS9zdG9yeS8yMDE2LzAyL3doeS1kb25h

...

article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiqAFodHRwczovL3JtaS5vcmcvcHJlc3MtcmVsZWFzZS9ybWktbGF1bmNoZXMtdGhlLWZpcnN0LWZpbmFuY2UtZnJhbWV3b3JrLXRvLXN1cHBvcnQtdGhlLWRlY2FyYm9uaXphdGlvbi1vZi1hbHVtaW51bS1wcm9kdWN0aW9uLXRoZS1zdXN0YWluYWJsZS1hbHVtaW51bS1maW5hbmNlLWZyYW1ld29yay_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://rmi.org/press-release/rmi-launches-the-first-finance-framework-to-support-the-decarbonization-of-aluminum-production-the-sustainable-aluminum-finance-framework/
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiTWh0dHBzOi8vd3d3LnVuZXAub3JnL3Jlc291cmNlcy9yZXBvcnQvc3RhdGUtZmluYW5jZS1uYXR1cmUtY2l0aWVzLXRpbWUtYXNzZXNz0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.unep.org/resources/report/state-finance-nature-cities-time-assess
article collected
article collected
Error occurred whi

article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiTmh0dHBzOi8vZGZwaS5jYS5nb3YvMjAyMy8wMy8yMy9maWxpbmctdGF4ZXMta2V5LXRvLW92ZXJhbGwtZmluYW5jaWFsLXdlbGxuZXNzL9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://dfpi.ca.gov/2023/03/23/filing-taxes-key-to-overall-financial-wellness/
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMidGh0dHBzOi8vd3d3Lm5qb2FnLmdvdi9hYm91dC9kaXZpc2lvbnMtYW5kLW9mZmljZXMvb2ZmaWNlLW9mLXNlY3VyaXRpZXMtZnJhdWQtYW5kLWZpbm

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiR2h0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjMvMTEvMTQvdXMvYXJpem9uYS1iaWRlbi12b3RlcnMtZWNvbm9teS5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.nytimes.com/2023/11/14/us/arizona-biden-voters-economy.html
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiSWh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjMvMTEvMDUvdXMvcG9saXRpY3MvYmlkZW4tdHJ1bXAtMjAyNC1wb2xsLmh0bWzSAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.nytimes.com/2023/11/05/us/politics/biden-trump-2024-poll.html
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBM

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiW2h0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9uZXdzLzIwMjMvMDgvMTgvYmlkZW5zLXJlZWxlY3Rpb24tZWNvbm9teS12b3RlcnMtb2YtY29sb3ItMDAxMTE3MzDSAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/news/2023/08/18/bidens-reelection-economy-voters-of-color-00111730
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiVGh0dHBzOi8vd3d3Lmlwc29zLmNvbS9lbi11cy9lY29ub215LWdldHRpbmctYmV0dGVyLXdoeS1pc250LWJpZGVuLWdldHRpbmctYW55LWNyZWRpdNIBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.ipsos.com/en-us/economy-getting-better-why-isnt-biden-getting-any-credit
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiVmh0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9uZXdzLzIwMjMvMTEvMjkvYmlkZW4taW5mbGF0aW9uLXNvZnQtbGFuZGluZy1lY29ub215LTAwMTI4OTk50gEA?oc=5&hl=en-US&gl=US&c

article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiQGh0dHBzOi8vd3d3LmJhcnJvbnMuY29tL2FydGljbGVzL3RydW1wLXZzLWJpZGVuLWVjb25vbXktZWM4ZmY4OWHSAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.barrons.com/articles/trump-vs-biden-economy-ec8ff89a
article collected
article collected
Error occurred during article extraction for URL: https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vd3d3LnBld3Jlc2VhcmNoLm9yZy9wb2xpdGljcy8yMDIzLzA0LzA3L2Fzc2Vzc21lbnRzLW9mLWJpZGVuLWFuZC1oaXMtYWRtaW5pc3RyYXRpb24v0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: Article `download()` failed with HTTPSConnectionPool(host='www.pewresearch.org', port=443): Read timed out. (read timeout=7) on URL https://news.google.com/rss/articles/CBMiXGh0dHBzOi8vd3d3LnBld3Jlc2VhcmNoLm9yZy9wb2xpdGljcy8yMDIzLzA0LzA3L2Fzc2Vzc21lbnRzLW9mLWJpZGVuLWFuZC1oaXMtYWRtaW5pc3RyYXRpb24v0gEA?oc=5&hl=en-US&gl=US&ceid=US:en
Error occ

In [31]:
start_date=(2016,1,1)
end_date=(2024,4,26)
keyword='economy'

pull_data(keyword, start_date, end_date)

article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiVGh0dHBzOi8vd3d3Lmdvdi5jYS5nb3YvMjAyNC8wNC8xNi9jYWxpZm9ybmlhLXJlbWFpbnMtdGhlLXdvcmxkcy01dGgtbGFyZ2VzdC1lY29ub215L9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.gov.ca.gov/2024/04/16/california-remains-the-worlds-5th-largest-economy/
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiRGh0dHBzOi8vd3d3LmFtZXJpY2FucHJvZ3Jlc3Mub3JnL2lzc3Vlcy9idWlsZGluZy1hbi1lY29ub215LWZvci1hbGwv0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.americanprogress.org/issues/building-an-economy-for-all/
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMijQFodHRwczovL3d3dy5zdGF0ZS5nb3YvYnVyZWF1cy1vZmZpY2VzL3VuZGVyLXNlY3JldGFyeS1mb3ItZWNvbm9taWMtZ3Jvd3RoLWVuZXJneS1hbmQtdGhlLWVu

In [31]:
df = pd.read_csv('data/econ_news.csv')
df

,Unnamed: 0,title,description,published date,url,source,extracted_content,author,urlToImage,publishedAt,content,source.id,source.name
0,0.0,"Circular economy: definition, importance and b...","Circular economy: definition, importance and b...","Wed, 24 May 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMidWh0d...,European Parliament,Benefits: why do we need to switch to a circul...,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,ICYMI: California Poised to Become World's 4th...,ICYMI: California Poised to Become World's 4th...,"Mon, 24 Oct 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMiX2h0d...,Office of Governor Gavin Newsom,ICYMI: California Poised to Become World's 4th...,NaN,NaN,NaN,NaN,NaN,NaN
2,2.0,A snapshot of a new working-from-home economy ...,A snapshot of a new working-from-home economy ...,"Mon, 29 Jun 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiR2h0d...,Stanford University News,"The new “working-from-home economy,” which is ...",NaN,NaN,NaN,NaN,NaN,NaN
3,3.0,Recreation Economy for Rural Communities | US ...,Recreation Economy for Rural Communities | US ...,"Wed, 06 Dec 2023 08:00:00 GMT",https://news.google.com/rss/articles/CBMiRGh0d...,U.S. EPA.gov,The Recreation Economy for Rural Communities p...,NaN,NaN,NaN,NaN,NaN,NaN
4,4.0,The Political Economy of Economic Policy - IMF...,The Political Economy of Economic Policy - IMF...,"Fri, 10 Jun 2022 22:51:41 GMT",https://news.google.com/rss/articles/CBMiamh0d...,International Monetary Fund,5 min (1403 words) Read\n\nDownload PDF\n\nWe ...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4119,NaN,'Bidenomics': Some House Democrats cool on Joe...,'Bidenomics': Some House Democrats cool on Joe...,"Tue, 29 Aug 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiemh0d...,USA TODAY,"STRATHAM, N.H. − While President Joe Biden tou...",NaN,NaN,NaN,NaN,NaN,NaN
4120,NaN,Yale professor: Biden's economy most successfu...,Yale professor: Biden's economy most successfu...,"Tue, 25 Jul 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiemh0d...,MSNBC,MSNBC's Lawrence O'Donnell is joined by Yale P...,NaN,NaN,NaN,NaN,NaN,NaN
4121,NaN,Bidenomics: White House adviser Jared Bernstei...,Bidenomics: White House adviser Jared Bernstei...,"Wed, 28 Jun 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMid2h0d...,NPR,Remember Reaganomics? Freakonomics? Now there'...,NaN,NaN,NaN,NaN,NaN,NaN
4122,NaN,The Big Squeeze: How Biden's Environmental Jus...,The Big Squeeze: How Biden's Environmental Jus...,"Thu, 07 Sep 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiaWh0d...,Manhattan Institute,Executive Summary\n\nFrom its first week in of...,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
# for rows that dont have extracted content- jion the title and description to make the extracted content 

def update_extracted_content(row):
    if pd.isna(row['extracted_content']) or row['extracted_content'] == '':
        return row['title'] + ' ' + row['description']
    else:
        return row['extracted_content']
    
df['extracted_content'] = df.apply(update_extracted_content, axis=1)
        
df.isna().sum()

# save as csv file 
#df.to_csv('data/econ_news.csv')

title                   0
description             0
published date        124
url                     0
source                124
extracted_content       0
author               1176
urlToImage           1176
publishedAt          1176
content              1176
source.id            1258
source.name          1176
dtype: int64

In [17]:
# save as csv file 
df.to_csv('data/econ_news.csv')

In [12]:
start_date=(2016,1,1)
end_date=(2024,4,25)

pull_data('finance', start_date, end_date)

article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiK2h0dHBzOi8vd3d3Lm10c3UuZWR1L3Byb2dyYW0vZmluYW5jZS1iLWItYS_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.mtsu.edu/program/finance-b-b-a/
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiLmh0dHBzOi8vZGFtb3JlLW1ja2ltLm5vcnRoZWFzdGVybi5lZHUvZmluYW5jZS_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://damore-mckim.northeastern.edu/finance/
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiqAFodHRwczovL2Jsb2dzLm1pY3Jvc29mdC5jb20vYmxvZy8yMDI0LzAyLzI5L2ludHJvZHVjaW5nLW1pY3Jvc29mdC1jb3BpbG90LWZvci1maW5hbmNlLXRoZS1uZXdlc3QtY29waWxvdC1vZmZlcmluZy1pbi1taWNyb3NvZnQtMzY1LWRlc2lnbmVkLXRvLXRyYW5zZm9ybS1tb2Rlcm4tZmluYW5jZS_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url:

04/25/2024 04:07:41 PM - newspaper ignoring lxml node error: Input object is not an XML element: HtmlProcessingInstruction


article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiRGh0dHBzOi8vd3d3LnVuLm9yZy9lbi9jbGltYXRlY2hhbmdlL3JhaXNpbmctYW1iaXRpb24vY2xpbWF0ZS1maW5hbmNl0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.un.org/en/climatechange/raising-ambition/climate-finance
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiQWh0dHBzOi8vd3d3Lm9lY2Qub3JnL2NsaW1hdGUtY2hhbmdlL2ZpbmFuY2UtdXNkLTEwMC1iaWxsaW9uLWdvYWwv0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.oecd.org/climate-change/finance-usd-100-billion-goal/
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiJ2h0dHBzOi8vd3d3LmNvcnZhbGxpc29yZWdvbi5nb3YvZmluYW5jZdIBAA?oc=5&hl=en-US&gl=US&

04/25/2024 04:09:52 PM - newspaper ignoring lxml node error: Input object is not an XML element: HtmlProcessingInstruction


article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiTmh0dHBzOi8vZGV2ZWxvcG1lbnQub2hpby5nb3YvYnVzaW5lc3Mvc3RhdGUtaW5jZW50aXZlcy90YXgtaW5jcmVtZW50LWZpbmFuY2luZ9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 404 Client Error: Not Found for url: https://development.ohio.gov/business/state-incentives/tax-increment-financing
article collected
article collected
Timeout occurred while fetching URL: https://news.google.com/rss/articles/CBMiIGh0dHBzOi8vb2x5bXBpY3MuY29tL2lvYy9maW5hbmNl0gEA?oc=5&hl=en-US&gl=US&ceid=US:en
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiaWh0dHBzOi8vd3d3LmNsaW1hdGVwb2xpY3lpbml0aWF0aXZlLm9yZy9wdWJsaWNhdGlvbi9nbG9iYWwtbGFuZHNjYXBlLW9mLWNsaW1hdGUtZmluYW5jZS1hLWRlY2FkZS1vZi1kYXRhL9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.climatepolicyinitiative.org/publication/globa

In [197]:
start_date=(2016,1,1)
end_date=(2024,4,25)

pull_data('financial', start_date, end_date)

article collected
article collected
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMihAFodHRwczovL3d3dy5wb2xpdGljby5jb20vbmV3cy8yMDI0LzAyLzAyL3RydW1wLWFuZC1oaXMtYWxsaWVzLWFkanVzdC10aGVpci1tZXNzYWdpbmctb24tdGhlLWVjb25vbXktYXMtaXQtc3RlYWRpbHktaW1wcm92ZXMtMDAxMzk0NDfSAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/news/2024/02/02/trump-and-his-allies-adjust-their-messaging-on-the-economy-as-it-steadily-improves-00139447
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiUmh0dHBzOi8vd3d3LmFtZXJpY2FucHJvZ3Jlc3Mub3JnL2FydGljbGUvdGhlLWJpZGVuLWJvb20tZWNvbm9taWMtcmVjb3ZlcnktaW4tMjAyMS_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.americanprogress.org/article/the-biden-boom-economic-

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiVmh0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9uZXdzLzIwMjMvMTEvMjkvYmlkZW4taW5mbGF0aW9uLXNvZnQtbGFuZGluZy1lY29ub215LTAwMTI4OTk50gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/news/2023/11/29/biden-inflation-soft-landing-economy-00128999
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiTWh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjMvMTEvMTcvYnVzaW5lc3MvZWNvbm9teS90aWt0b2stYmlkZW4tZWNvbm9teS5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.nytimes.com/2023/11/17/business/economy/tiktok-biden-economy.html
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiP2h0dHBzOi8vd3d3LmF4aW9zLmNvbS8yMDI0LzAxLzE0L2JpZGVuLWVjb25vbXktbWVzc2FnZS1lbGVjdGlvbtIBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.axios.com/2024/01/14/biden-e

In [199]:
start_date=(2016,1,1)
end_date=(2020,1,1)

pull_data('economy', start_date, end_date)

article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMijQFodHRwczovL3d3dy5zdGF0ZS5nb3YvYnVyZWF1cy1vZmZpY2VzL3VuZGVyLXNlY3JldGFyeS1mb3ItZWNvbm9taWMtZ3Jvd3RoLWVuZXJneS1hbmQtdGhlLWVudmlyb25tZW50L2J1cmVhdS1vZi1lY29ub21pYy1hbmQtYnVzaW5lc3MtYWZmYWlycy_SAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.state.gov/bureaus-offices/under-secretary-for-economic-growth-energy-and-the-environment/bureau-of-economic-and-business-affairs/
article collected
article collected
article collected
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiLGh0dHBzOi8vd3d3LnJlbm8uZ292L2J1c2luZXNzL3Jlbm8tcy1lY29ub2150gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.reno.gov/business/reno-s-economy
article collected
article collected
article collected
article collected
article collec

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiY2h0dHBzOi8vd3d3LmFtZXJpY2FucHJvZ3Jlc3Mub3JnL2FydGljbGUvcmFpc2luZy1wYXktcHJvdmlkaW5nLWJlbmVmaXRzLXdvcmtlcnMtZGlzcnVwdGl2ZS1lY29ub215L9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.americanprogress.org/article/raising-pay-providing-benefits-workers-disruptive-economy/
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiSWh0dHBzOi8vd3d3LnVuLm9yZy9kZXZlbG9wbWVudC9kZXNhL2VuL25ld3Mvc3VzdGFpbmFibGUvYmx1ZS1lY29ub215Lmh0bWzSAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.un.org/development/desa/en/news/sustainable/blue-economy.html
results saved to data/econ_news.csv


In [13]:
test_df = pd.read_csv('data/econ_news.csv')
test_df

,title,description,published date,url,source,extracted_content,author,urlToImage,publishedAt,content,source.id,source.name
0,"Circular economy: definition, importance and b...","Circular economy: definition, importance and b...","Wed, 24 May 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMidWh0d...,European Parliament,Benefits: why do we need to switch to a circul...,NaN,NaN,NaN,NaN,NaN,NaN
1,ICYMI: California Poised to Become World's 4th...,ICYMI: California Poised to Become World's 4th...,"Mon, 24 Oct 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMiX2h0d...,Office of Governor Gavin Newsom,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A snapshot of a new working-from-home economy ...,A snapshot of a new working-from-home economy ...,"Mon, 29 Jun 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiR2h0d...,Stanford University News,"The new “working-from-home economy,” which is ...",NaN,NaN,NaN,NaN,NaN,NaN
3,Recreation Economy for Rural Communities | US ...,Recreation Economy for Rural Communities | US ...,"Wed, 06 Dec 2023 08:00:00 GMT",https://news.google.com/rss/articles/CBMiRGh0d...,U.S. EPA.gov,The Recreation Economy for Rural Communities p...,NaN,NaN,NaN,NaN,NaN,NaN
4,The Political Economy of Economic Policy - IMF...,The Political Economy of Economic Policy - IMF...,"Fri, 10 Jun 2022 22:51:41 GMT",https://news.google.com/rss/articles/CBMiamh0d...,International Monetary Fund,5 min (1403 words) Read\n\nDownload PDF\n\nWe ...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
828,Turning the Tide: How to Finance a Sustainable...,Turning the Tide: How to Finance a Sustainable...,"Mon, 01 Mar 2021 17:20:38 GMT",https://news.google.com/rss/articles/CBMiNWh0d...,UNEP Finance Initiative,A practical guide for financial institutions t...,NaN,NaN,NaN,NaN,NaN,NaN
829,Virginia Dept. of Elections: Campaign Finance ...,Virginia Dept. of Elections: Campaign Finance ...,"Mon, 02 Oct 2017 04:16:21 GMT",https://news.google.com/rss/articles/CBMiP2h0d...,News Releases - Virginia Department of Elections,Campaign Finance Reports\n\nOnce a committee h...,NaN,NaN,NaN,NaN,NaN,NaN
830,UNSDG | People's Money: Harnessing Digitalizat...,UNSDG | People's Money: Harnessing Digitalizat...,"Fri, 28 Aug 2020 05:41:19 GMT",https://news.google.com/rss/articles/CBMiYWh0d...,United Nations Sustainable Development Group,"Announcement\n\n07 February 2024\n\n""Peace can...",NaN,NaN,NaN,NaN,NaN,NaN
831,"Finance Definition: Finance Industry, Skills &...","Finance Definition: Finance Industry, Skills &...","Wed, 19 Oct 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMiG2h0d...,Built In,Finance Career Paths\n\nImage: Shutterstock\n\...,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
start_date=(2016,1,1)
end_date=(2024,4,26)

pull_data('Trump economy', start_date, end_date)

article collected
article collected
Timeout occurred while fetching URL: https://news.google.com/rss/articles/CBMic2h0dHBzOi8vd3d3LnVzbmV3cy5jb20vbmV3cy9lY29ub215L2FydGljbGVzLzIwMjQtMDMtMTUvd2hhdC10aGUtdS1zLWVjb25vbXktd291bGQtbG9vay1saWtlLWluLWEtc2Vjb25kLXRydW1wLXRlcm3SAQA?oc=5&hl=en-US&gl=US&ceid=US:en
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiVmh0dHBzOi8vZmluYW5jZS55YWhvby5jb20vbmV3cy93aHktYW1lcmljYW5zLXBpbmUtZm9yLXRoZS10cnVtcC1lY29ub215LTE5Mzc1NzgxNS5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 404 Client Error: Not Found for url: https://finance.yahoo.com/news/why-americans-pine-for-the-trump-economy-193757815.html
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMihAFodHRwczovL3d3dy5wb2xpdGljby5jb20vbmV3cy8yMDI0LzAyLzAyL3RydW1wLWFuZC1oaXMtYWxsaWVzLWFkanVzdC10aGVpci1tZXNzYWdpbmctb24tdGhlLWVjb25vbXktYXMtaXQtc3RlYWR

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiRGh0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDEvMTUvb3Bpbmlvbi90cnVtcC12b3RlcnMtZWNvbm9teS5odG1s0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.nytimes.com/2024/01/15/opinion/trump-voters-economy.html
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiU2h0dHBzOi8vd3d3LmFtZXJpY2FucHJvZ3Jlc3Mub3JnL2FydGljbGUvcHJlc2lkZW50LXRydW1wLXdvcmtlcnMtY29udGludWUtc3RydWdnbGUv0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.americanprogress.org/article/president-trump-workers-continue-struggle/
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiUGh0dHBzOi8vd3d3LmFtZXJpY2FucHJvZ3Jlc3Mub3JnL2FydGljbGUvdHJ1bXBzLWNvcnBvcmF0ZS10YXgtY3V0LW5vdC10cmlja2xpbmcv0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. 

In [15]:
start_date=(2016,1,1)
end_date=(2024,4,26)

pull_data('Biden economy', start_date, end_date)

article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMihAFodHRwczovL3d3dy5wb2xpdGljby5jb20vbmV3cy8yMDI0LzAyLzAyL3RydW1wLWFuZC1oaXMtYWxsaWVzLWFkanVzdC10aGVpci1tZXNzYWdpbmctb24tdGhlLWVjb25vbXktYXMtaXQtc3RlYWRpbHktaW1wcm92ZXMtMDAxMzk0NDfSAQA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/news/2024/02/02/trump-and-his-allies-adjust-their-messaging-on-the-economy-as-it-steadily-improves-00139447
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiYGh0dHBzOi8vd3d3LnJldXRlcnMuY29tL3dvcmxkL3VzL29iYW1hLWNsaW50b24td2h5LWFtZXJpY2Fucy1kb250LWxvdmUtYmlkZW4tZWNvbm9teS0yMDI0LTAzLTI5L9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 401 Client Error: HTTP Forbidden for url: https://www.reuters.com/world/us/obama-clinton-why-americans-dont-love-biden-economy-2024-03-29/
article collected
article collecte

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiV2h0dHBzOi8vd3d3Lm55dGltZXMuY29tLzIwMjQvMDMvMDUvYnVzaW5lc3MvZWNvbm9teS9iaWRlbi1lY29ub215LXRpbWVzLXNpZW5hLXBvbGwuaHRtbNIBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.nytimes.com/2024/03/05/business/economy/biden-economy-times-siena-poll.html
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiaWh0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9uZXdzLzIwMjQvMDEvMjUvYmlkZW4tYWRtaW5pc3RyYXRpb24tZWNvbm9taWMtZ3Jvd3RoLWluZmxhdGlvbi1kZWNyZWFzZS0wMDEzNzc3N9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/news/2024/01/25/biden-administration-economic-growth-inflation-decrease-00137777
article collected
article collected
article collected
article collected
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiY2h0dHBzOi8vd3d3LnJldXRlcnMuY29tL2JyZWFraW5ndmll

In [16]:
econ_news = pd.read_csv('data/econ_news.csv')
econ_news

,title,description,published date,url,source,extracted_content,author,urlToImage,publishedAt,content,source.id,source.name
0,"Circular economy: definition, importance and b...","Circular economy: definition, importance and b...","Wed, 24 May 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMidWh0d...,European Parliament,Benefits: why do we need to switch to a circul...,NaN,NaN,NaN,NaN,NaN,NaN
1,ICYMI: California Poised to Become World's 4th...,ICYMI: California Poised to Become World's 4th...,"Mon, 24 Oct 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMiX2h0d...,Office of Governor Gavin Newsom,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A snapshot of a new working-from-home economy ...,A snapshot of a new working-from-home economy ...,"Mon, 29 Jun 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiR2h0d...,Stanford University News,"The new “working-from-home economy,” which is ...",NaN,NaN,NaN,NaN,NaN,NaN
3,Recreation Economy for Rural Communities | US ...,Recreation Economy for Rural Communities | US ...,"Wed, 06 Dec 2023 08:00:00 GMT",https://news.google.com/rss/articles/CBMiRGh0d...,U.S. EPA.gov,The Recreation Economy for Rural Communities p...,NaN,NaN,NaN,NaN,NaN,NaN
4,The Political Economy of Economic Policy - IMF...,The Political Economy of Economic Policy - IMF...,"Fri, 10 Jun 2022 22:51:41 GMT",https://news.google.com/rss/articles/CBMiamh0d...,International Monetary Fund,5 min (1403 words) Read\n\nDownload PDF\n\nWe ...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
902,Hearing Wrap Up: The Biden Administration's Re...,Hearing Wrap Up: The Biden Administration's Re...,"Wed, 14 Jun 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMihQFod...,House Committee on Oversight and Reform |,"WASHINGTON – Today, the House Committee on Ove...",NaN,NaN,NaN,NaN,NaN,NaN
903,US voters warm to Joe Biden on economy but rem...,US voters warm to Joe Biden on economy but rem...,"Mon, 15 Apr 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMiP2h0d...,Financial Times,Try unlimited access Only $1 for 4 weeks\n\nTh...,NaN,NaN,NaN,NaN,NaN,NaN
904,"Biden gets low ratings on economy, guns, immig...","Biden gets low ratings on economy, guns, immig...","Mon, 22 May 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMiZGh0d...,The Associated Press,"</script><p class=""social-caption""> </p></div>...",NaN,NaN,NaN,NaN,NaN,NaN
905,Analysis | Fox News twists data on economy and...,Analysis | Fox News twists data on economy and...,"Tue, 09 Apr 2024 07:00:00 GMT",https://news.google.com/rss/articles/CBMiRWh0d...,The Washington Post,There is an enduring chicken-egg question in A...,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
econ_news['url'][902]

'https://news.google.com/rss/articles/CBMihQFodHRwczovL292ZXJzaWdodC5ob3VzZS5nb3YvcmVsZWFzZS9oZWFyaW5nLXdyYXAtdXAtdGhlLWJpZGVuLWFkbWluaXN0cmF0aW9ucy1yZWd1bGF0b3J5LWJsaXR6LWplb3BhcmRpemVzLWFtZXJpY2FzLWVjb25vbXklRUYlQkYlQkMv0gEA?oc=5&hl=en-US&gl=US&ceid=US:en'

In [211]:
econ_news.isna().sum()

title                  0
description            0
published date         0
url                    0
source                 0
extracted_content    114
dtype: int64

## Pulling content using beautifulsoup 



In [202]:
import requests
from bs4 import BeautifulSoup 

In [203]:
def extract_article_text_bsoup(url):
    try:
        response = requests.get(url, timeout=30)  # Set timeout to 30 seconds
        response.raise_for_status()
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # find all <p> tags and extract their text
            paragraphs = soup.find_all('p')
            article_text = '/n'.join([p.get_text() for p in paragraphs])
            print("article collected")
            return article_text
        else:
            print(f"Failed to fetch URL: {url}")
            return None
    except requests.Timeout:
        print(f"Timeout occurred while fetching URL: {url}")
        return None
    except requests.RequestException as e:
        print(f"Error occurred while fetching URL: {url}. Error: {e}")
        return None

In [205]:
for index, row in econ_news.iterrows():
    if pd.isna(row['extracted_content']):
        econ_news.at[index, 'extracted_content'] = extract_article_newspaper(row['url'])

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiX2h0dHBzOi8vd3d3Lmdvdi5jYS5nb3YvMjAyMi8xMC8yNC9pY3ltaS1jYWxpZm9ybmlhLXBvaXNlZC10by1iZWNvbWUtd29ybGRzLTR0aC1iaWdnZXN0LWVjb25vbXkv0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.gov.ca.gov/2022/10/24/icymi-california-poised-to-become-worlds-4th-biggest-economy/
article collected
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiV2h0dHBzOi8vd3d3Lm9lY2Qub3JnL2RpZ2l0YWwvaWVjb25vbXkvb2VjZC1kaWdpdGFsLWVjb25vbXktb3V0bG9vay0yMDIwLWJiMTY3MDQxLWVuLmh0bdIBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.oecd.org/digital/ieconomy/oecd-digital-economy-outlook-2020-bb167041-en.htm
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMifWh0dHBzOi8vd3d3LndlZm9ydW0ub3JnL3B1YmxpY2F0aW9ucy9uYXR1cmUtcmlzay1yaXNpbmctd2h5LXRoZS1jcmlzaXMtZW5ndWxmaW5nLW5hdHVyZS1tYXR0ZXJzLWZvci1idXNpbmVzcy1hbmQtdGhlLWVjb2

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiVGh0dHBzOi8vd3d3Lmlwc29zLmNvbS9lbi11cy9lY29ub215LWdldHRpbmctYmV0dGVyLXdoeS1pc250LWJpZGVuLWdldHRpbmctYW55LWNyZWRpdNIBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.ipsos.com/en-us/economy-getting-better-why-isnt-biden-getting-any-credit
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMifmh0dHBzOi8vd3d3Lndzai5jb20vcG9saXRpY3MvZWxlY3Rpb25zL3ZvdGVycy1hcmUtbW9yZS11cGJlYXQtb24tZWNvbm9teS1idXQtYmlkZW4tZ2V0cy1saXR0bGUtYmVuZWZpdC13c2otcG9sbC1zaG93cy04ZTYwOGRiONIBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.wsj.com/politics/elections/voters-are-more-upbeat-on-economy-but-biden-gets-little-benefit-wsj-poll-shows-8e608db8
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiW2h0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9uZXdzLzIwMjMvMDgvMTgvYmlkZW5zLXJlZWxlY3Rpb24tZWNvbm9teS12b3RlcnMtb2YtY29sb3ItMD

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiSmh0dHBzOi8vd3d3LmJscy5nb3YvY2FyZWVyb3V0bG9vay8yMDE2L2FydGljbGUvd2hhdC1pcy10aGUtZ2lnLWVjb25vbXkuaHRt0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.bls.gov/careeroutlook/2016/article/what-is-the-gig-economy.htm
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiSGh0dHBzOi8vd3d3LndlZm9ydW0ub3JnL2FnZW5kYS8yMDE3LzAzL3dvcmxkcy1iaWdnZXN0LWVjb25vbWllcy1pbi0yMDE3L9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.weforum.org/agenda/2017/03/worlds-biggest-economies-in-2017/
Error occurred during article extraction for URL: https://news.google.com/rss/articles/CBMidWh0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvam9obmZyYXplcjEvMjAxOS8wMi8xNS9ob3ctdGhlLWdpZy1lY29ub215LWlzLXJlc2hhcGluZy1jYXJlZXJzLWZvci10aGUtbmV4dC1nZW5lcmF0aW9uL9IBAA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: Article `download()` failed with 403 Client Err

Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiTWh0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9zdG9yeS8yMDE2LzA5L2RvbmFsZC10cnVtcC1qb2JzLWVjb25vbWljLXBsYW4tMjI4MjE40gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/story/2016/09/donald-trump-jobs-economic-plan-228218
Error occurred while fetching URL: https://news.google.com/rss/articles/CBMiYmh0dHBzOi8vd3d3LnBvbGl0aWNvLmNvbS9uZXdzLzIwMTkvMTAvMTUvd2FybmluZy10by1kZW1vY3JhdHMtZWNvbm9teS1wb2ludHMtdG8tYS10cnVtcC13aW4tMDQ3MDIx0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: 403 Client Error: Forbidden for url: https://www.politico.com/news/2019/10/15/warning-to-democrats-economy-points-to-a-trump-win-047021
Error occurred during article extraction for URL: https://news.google.com/rss/articles/CBMiX2h0dHBzOi8vd3d3LmZvcmJlcy5jb20vc2l0ZXMvY2h1Y2tqb25lcy8yMDE5LzAzLzE2L2ZhY3QtY2hlY2tpbmctdHJ1bXBzLXJlY2VudC1lY29ub21pYy10d2VldHMv0gEA?oc=5&hl=en-US&gl=US&ceid=US:en. Error: Article `d

In [206]:
econ_news

,title,description,published date,url,source,extracted_content
0,"Circular economy: definition, importance and b...","Circular economy: definition, importance and b...","Wed, 24 May 2023 07:00:00 GMT",https://news.google.com/rss/articles/CBMidWh0d...,European Parliament,Benefits: why do we need to switch to a circul...
1,ICYMI: California Poised to Become World's 4th...,ICYMI: California Poised to Become World's 4th...,"Mon, 24 Oct 2022 07:00:00 GMT",https://news.google.com/rss/articles/CBMiX2h0d...,Office of Governor Gavin Newsom,None
2,A snapshot of a new working-from-home economy ...,A snapshot of a new working-from-home economy ...,"Mon, 29 Jun 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiR2h0d...,Stanford University News,"The new “working-from-home economy,” which is ..."
3,Recreation Economy for Rural Communities | US ...,Recreation Economy for Rural Communities | US ...,"Wed, 06 Dec 2023 08:00:00 GMT",https://news.google.com/rss/articles/CBMiRGh0d...,U.S. EPA.gov,The Recreation Economy for Rural Communities p...
4,The Political Economy of Economic Policy - IMF...,The Political Economy of Economic Policy - IMF...,"Fri, 10 Jun 2022 22:51:41 GMT",https://news.google.com/rss/articles/CBMiamh0d...,International Monetary Fund,5 min (1403 words) Read\n\nDownload PDF\n\nWe ...
...,...,...,...,...,...,...
392,Trump's maximum pressure campaign hammers Iran...,Trump's maximum pressure campaign hammers Iran...,"Sun, 14 Apr 2019 07:00:00 GMT",https://news.google.com/rss/articles/CBMiQ2h0d...,Axios,None
393,"Trump Mixes Economic and National Security, Pl...","Trump Mixes Economic and National Security, Pl...","Sat, 08 Jun 2019 07:00:00 GMT",https://news.google.com/rss/articles/CBMiUGh0d...,The New York Times,None
394,The president tweeted it – but is the US econo...,The president tweeted it – but is the US econo...,"Sat, 05 Aug 2017 07:00:00 GMT",https://news.google.com/rss/articles/CBMicGh0d...,The Guardian,It was a boast that every political leader wou...
395,How Trump's sanctions are crippling Iran's eco...,How Trump's sanctions are crippling Iran's eco...,"Mon, 24 Jun 2019 07:00:00 GMT",https://news.google.com/rss/articles/CBMiUWh0d...,DW (English),The US move to reimpose sanctions on Iran and ...


## Preprocessing 

In [171]:
poop = pd.read_csv('data/extracted_biden_news.csv')
poop.isna().sum()

poop.dropna(subset=['extracted_content'], inplace=True)
poop.isna().sum()

poop.to_csv('data/extracted_biden_news.csv')

In [165]:
# get rid of punctuation 



Unnamed: 0           0
title                0
description          0
published date       0
url                  0
source               0
extracted_content    0
dtype: int64

In [149]:
poop_separated = pd.read_csv('data/biden_news_separated.csv')
poop_separated

,title,description,published date,url,source,extracted_content
0,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,Even with a new president and political party ...
1,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,"Most notably, the stark economic rift that Bro..."
2,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,A similar analysis for last week’s election sh...
3,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,Table 1. Candidates’ counties won and share of...
4,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,Year Candidate Counties won Total votes Aggreg...
...,...,...,...,...,...,...
9128,Opinion | Biden embraced Trump's tariffs. It m...,Opinion | Biden embraced Trump's tariffs. It m...,"Fri, 12 Apr 2024 16:36:00 GMT",https://news.google.com/rss/articles/CBMiWWh0d...,The Washington Post,Follow this author Fareed Zakaria 's opinions ...
9129,Opinion | Biden embraced Trump's tariffs. It m...,Opinion | Biden embraced Trump's tariffs. It m...,"Fri, 12 Apr 2024 16:36:00 GMT",https://news.google.com/rss/articles/CBMiWWh0d...,The Washington Post,The Office of the U.S. Trade Representative pr...
9130,Opinion | Biden embraced Trump's tariffs. It m...,Opinion | Biden embraced Trump's tariffs. It m...,"Fri, 12 Apr 2024 16:36:00 GMT",https://news.google.com/rss/articles/CBMiWWh0d...,The Washington Post,Advertisement
9131,Opinion | Biden embraced Trump's tariffs. It m...,Opinion | Biden embraced Trump's tariffs. It m...,"Fri, 12 Apr 2024 16:36:00 GMT",https://news.google.com/rss/articles/CBMiWWh0d...,The Washington Post,There is also a foreign policy cost to the ris...


In [144]:
biden_news_df

,title,description,published date,url,source,extracted_content
1,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,Even with a new president and political party ...
1,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,"Most notably, the stark economic rift that Bro..."
1,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,A similar analysis for last week’s election sh...
1,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,Table 1. Candidates’ counties won and share of...
1,Biden-voting counties equal 70% of America's e...,Biden-voting counties equal 70% of America's e...,"Tue, 10 Nov 2020 08:00:00 GMT",https://news.google.com/rss/articles/CBMilAFod...,Brookings Institution,Year Candidate Counties won Total votes Aggreg...
...,...,...,...,...,...,...
499,Opinion | Biden embraced Trump's tariffs. It m...,Opinion | Biden embraced Trump's tariffs. It m...,"Fri, 12 Apr 2024 16:36:00 GMT",https://news.google.com/rss/articles/CBMiWWh0d...,The Washington Post,Follow this author Fareed Zakaria 's opinions ...
499,Opinion | Biden embraced Trump's tariffs. It m...,Opinion | Biden embraced Trump's tariffs. It m...,"Fri, 12 Apr 2024 16:36:00 GMT",https://news.google.com/rss/articles/CBMiWWh0d...,The Washington Post,The Office of the U.S. Trade Representative pr...
499,Opinion | Biden embraced Trump's tariffs. It m...,Opinion | Biden embraced Trump's tariffs. It m...,"Fri, 12 Apr 2024 16:36:00 GMT",https://news.google.com/rss/articles/CBMiWWh0d...,The Washington Post,Advertisement
499,Opinion | Biden embraced Trump's tariffs. It m...,Opinion | Biden embraced Trump's tariffs. It m...,"Fri, 12 Apr 2024 16:36:00 GMT",https://news.google.com/rss/articles/CBMiWWh0d...,The Washington Post,There is also a foreign policy cost to the ris...


In [145]:
trump_news_df

,title,description,published date,url,source,extracted_content
2,How the Trump economy compares to economies un...,How the Trump economy compares to economies un...,"Wed, 28 Oct 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiVmh0d...,CNN,AMERICA’S ECONOMY These 10 charts show how the...
2,How the Trump economy compares to economies un...,How the Trump economy compares to economies un...,"Wed, 28 Oct 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiVmh0d...,CNN,At the start of Donald Trump’s presidency in J...
2,How the Trump economy compares to economies un...,How the Trump economy compares to economies un...,"Wed, 28 Oct 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiVmh0d...,CNN,Employers had added jobs for 76 months straigh...
2,How the Trump economy compares to economies un...,How the Trump economy compares to economies un...,"Wed, 28 Oct 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiVmh0d...,CNN,Then came the pandemic.
2,How the Trump economy compares to economies un...,How the Trump economy compares to economies un...,"Wed, 28 Oct 2020 07:00:00 GMT",https://news.google.com/rss/articles/CBMiVmh0d...,CNN,"Below, we’ve tracked 10 indicators to show how..."
...,...,...,...,...,...,...
499,"If America falls into recession, Donald Trump ...","If America falls into recession, Donald Trump ...","Wed, 08 Jun 2016 07:00:00 GMT",https://news.google.com/rss/articles/CBMiXGh0d...,The Week,That's essentially the same conclusion from JP...
499,"If America falls into recession, Donald Trump ...","If America falls into recession, Donald Trump ...","Wed, 08 Jun 2016 07:00:00 GMT",https://news.google.com/rss/articles/CBMiXGh0d...,The Week,"The risk of recession is increasing, Still, it..."
499,"If America falls into recession, Donald Trump ...","If America falls into recession, Donald Trump ...","Wed, 08 Jun 2016 07:00:00 GMT",https://news.google.com/rss/articles/CBMiXGh0d...,The Week,But here's the thing none of these models fact...
499,"If America falls into recession, Donald Trump ...","If America falls into recession, Donald Trump ...","Wed, 08 Jun 2016 07:00:00 GMT",https://news.google.com/rss/articles/CBMiXGh0d...,The Week,"For instance, in a recent op-ed, former Treasu..."
